In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy 
import seaborn as sns
sns.set_style("darkgrid")

# Energy lost per unit time by the neutron star

mL = 1.115 # mass if lambda baryon in GeV

# Crossection as function of momentum
def sigmav(p):
    # Lets consider the unsuppressed operator O4 from pp-annihilation paper
    sigv = (mL**2 + p**2) * np.sqrt(p**2 + mL**2) / (4 * np.pi * mL)
    return sigv

# Averaging cross-section over the Fermi Sea
def avg_sigmav(n):
    return scipy.integrate.quad(sigmav, 0, (3 * np.pi ** 2 * n) ** (1/3))[0] / (3 * np.pi ** 2 * n) ** (1/3) 

In [ ]:
# Interpolation of number density function

raw_nd = np.transpose(np.array(pd.read_csv("nd(fm-3)_r(km).csv", header=None)))

r_nd, index_nd = np.unique(raw_nd[0], return_index=True)
nd = raw_nd[1, index_nd]

# Properly multiplying by the conversion factor so that everything is in GeV
r_nd = (1 / 0.197) * 10 ** 18 * r_nd
nd = 0.197 ** 3 * nd

# Defining the interpolated function for the number density, and setting values outside the domain of interest to be zero
numden = scipy.interpolate.interp1d(r_nd, nd, kind='quadratic', fill_value=0, bounds_error=False)

In [ ]:
# Intepolating the Energy function

raw_en = np.transpose(np.array(pd.read_csv("en(GeV)_r(km).csv", header=None)))

r_en, index_en = np.unique(raw_en[0], return_index=True)
en = raw_en[1, index_en]

# Properly multiplying by the conversion factor so that everything is in GeV
r_en = (1 / 0.197) * 10 ** 18 * r_en

# Defining the interpolated function for the energy, and setting values outside the domain of interest to be zero
energy = scipy.interpolate.interp1d(r_en, en, kind='quadratic', fill_value=0, bounds_error=False)


In [ ]:
# Computing the total energy lost per unit time in the entire neutron star

r_max = max(r_nd)
r_min = min(r_nd)
def integrand(r):
    return 4 * np.pi * r ** 2 * numden(r) ** 2 * energy(r) * avg_sigmav(numden(r))

Rate= scipy.integrate.quad(integrand, r_min, r_max)[0]

In [ ]:
def num_lambda(r):
    return 4 *  np.pi * numden(r) * r ** 2

Num_of_lambda = scipy.integrate.quadrature(num_lambda, r_min, r_max)

print(f"No of Hyperons in the neutron star : {Num_of_lambda[0]}")

In [ ]:
# Rate of energy lost by coldest neutron star
rate_neutron_star = 4 * np.pi * (11 * 10 ** 3) **2 * 5.67 * 10 ** (-8) * (42000) ** 4 * 6.242 * 10 ** 9 * 6.58 * 10 ** (-25) # In  GeV^2

# Defining Lambda 
Lambda =  (Rate / rate_neutron_star) ** (1/4)
print("Lambda : {:.2e}".format(Lambda))
